In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression

In [2]:
df = pd.read_pickle("../data/interim/country_info.gpickle")
df

,country_name,country_code,total_pop,arrivals,departures,latitude,longitude,arrivals/total,departures/total,arrivals/population,departures/population,continent_name,continent_code,destinations
0,Afghanistan,AFG,37172386.0,NaN,NaN,33.0000,65.0000,NaN,NaN,NaN,NaN,Asia,AS,"[IND, PAK, ARE, IRN, TUR, AZE, KWT, TJK]"
1,Albania,ALB,2866376.0,5340000.0,5415000.0,41.0000,20.0000,0.003759,0.004280,1.862980,1.889145,Europe,EU,"[GRC, ITA, GBR, DEU, SVN, AUT, TUR]"
2,Algeria,DZA,42228429.0,2657000.0,5610000.0,28.0000,3.0000,0.001870,0.004434,0.062920,0.132849,Africa,AF,"[DEU, FRA, TUR, CIV, ESP, JOR, LBN, MLI, BEL, ..."
3,American Samoa,ASM,64948.0,20200.0,NaN,-14.3333,-170.0000,0.000014,NaN,0.311018,NaN,Oceania,OC,[USA]
4,Andorra,AND,1198575.0,3042000.0,NaN,42.5000,1.6000,0.002141,NaN,2.538014,NaN,Europe,EU,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213,Virgin Islands (U.S.),VIR,106977.0,381000.0,NaN,18.3333,-64.8333,0.000268,NaN,3.561513,NaN,North America,NaN,"[VGB, PRI, USA, AIA, DMA, KNA]"
214,West Bank and Gaza,PSE,4569087.0,606000.0,NaN,32.0000,35.2500,0.000427,NaN,0.132630,NaN,Asia,AS,NaN
215,"Yemen, Rep.",YEM,28498687.0,366700.0,NaN,15.0000,48.0000,0.000258,NaN,0.012867,NaN,Asia,AS,"[ARE, SAU, SOM, DJI, BHR, JOR, IND, EGY, QAT, ..."
216,Zambia,ZMB,17351822.0,1072000.0,NaN,-15.0000,30.0000,0.000755,NaN,0.061780,NaN,Africa,AF,"[NLD, ZWE, MWI, KEN, ZAF, BWA, AGO, ARE, ETH, ..."


In [3]:
df.loc[df["country_name"] == "Spain"]

,country_name,country_code,total_pop,arrivals,departures,latitude,longitude,arrivals/total,departures/total,arrivals/population,departures/population,continent_name,continent_code,destinations
180,Spain,ESP,46723749.0,82773000.0,19116000.0,40.0,-4.0,0.058266,0.015108,1.77154,0.409128,Europe,EU,"[CHE, GMB, MAR, DEU, LVA, GRC, GBR, USA, AUT, ..."


In [4]:
df.isna().sum()

country_name              0
country_code              0
total_pop                 0
arrivals                 10
departures               82
latitude                  0
longitude                 0
arrivals/total           10
departures/total         82
arrivals/population      10
departures/population    82
continent_name            0
continent_code           31
destinations              6
dtype: int64

# AÑADO POPULATION DE OTRA FUENTE DE DATOS MAS COMPLETA

In [5]:
df_population = pd.read_csv("../data/raw/country_population.csv", skiprows=4)
df_population

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,Unnamed: 64
0,Aruba,ABW,"Population, total",SP.POP.TOTL,54211.0,55438.0,56225.0,56695.0,57032.0,57360.0,...,102046.0,102560.0,103159.0,103774.0,104341.0,104872.0,105366.0,105845.0,106314.0,NaN
1,Afghanistan,AFG,"Population, total",SP.POP.TOTL,8996973.0,9169410.0,9351441.0,9543205.0,9744781.0,9956320.0,...,30117413.0,31161376.0,32269589.0,33370794.0,34413603.0,35383128.0,36296400.0,37172386.0,38041754.0,NaN
2,Angola,AGO,"Population, total",SP.POP.TOTL,5454933.0,5531472.0,5608539.0,5679458.0,5735044.0,5770570.0,...,24220661.0,25107931.0,26015780.0,26941779.0,27884381.0,28842484.0,29816748.0,30809762.0,31825295.0,NaN
3,Albania,ALB,"Population, total",SP.POP.TOTL,1608800.0,1659800.0,1711319.0,1762621.0,1814135.0,1864791.0,...,2905195.0,2900401.0,2895092.0,2889104.0,2880703.0,2876101.0,2873457.0,2866376.0,2854191.0,NaN
4,Andorra,AND,"Population, total",SP.POP.TOTL,13411.0,14375.0,15370.0,16412.0,17469.0,18549.0,...,83747.0,82427.0,80774.0,79213.0,78011.0,77297.0,77001.0,77006.0,77142.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,Kosovo,XKX,"Population, total",SP.POP.TOTL,947000.0,966000.0,994000.0,1022000.0,1050000.0,1078000.0,...,1791000.0,1807106.0,1818117.0,1812771.0,1788196.0,1777557.0,1791003.0,1797085.0,1794248.0,NaN
260,"Yemen, Rep.",YEM,"Population, total",SP.POP.TOTL,5315355.0,5393036.0,5473671.0,5556766.0,5641597.0,5727751.0,...,23807588.0,24473178.0,25147109.0,25823485.0,26497889.0,27168210.0,27834821.0,28498687.0,29161922.0,NaN
261,South Africa,ZAF,"Population, total",SP.POP.TOTL,17099840.0,17524533.0,17965725.0,18423161.0,18896307.0,19384841.0,...,52004172.0,52834005.0,53689236.0,54545991.0,55386367.0,56203654.0,57000451.0,57779622.0,58558270.0,NaN
262,Zambia,ZMB,"Population, total",SP.POP.TOTL,3070776.0,3164329.0,3260650.0,3360104.0,3463213.0,3570464.0,...,14023193.0,14465121.0,14926504.0,15399753.0,15879361.0,16363507.0,16853688.0,17351822.0,17861030.0,NaN


Compruebo la diferencia 

In [6]:
country = "Spain"
print(df.loc[df["country_name"] == country, "total_pop"])
print(df_population.loc[df_population["Country Name"] == country, "2019"])

180    46723749.0
Name: total_pop, dtype: float64
68    47076781.0
Name: 2019, dtype: float64


Miro si estoy cogiendo el ultimo año disponible en el dataframe que tenía:

In [7]:
df_world = pd.read_csv("../data/raw/world_indicators_data.csv")
df_world.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,Unnamed: 64
0,Arab World,ARB,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,82.783289,83.120303,83.533457,83.897596,84.171599,84.510171,NaN,NaN,NaN,NaN
1,Arab World,ARB,Access to electricity (% of population),EG.ELC.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,86.428272,87.070576,88.176836,87.342739,89.130121,89.678685,90.273687,NaN,NaN,NaN
2,Arab World,ARB,"Access to electricity, rural (% of rural popul...",EG.ELC.ACCS.RU.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,73.942103,75.244104,77.162305,75.538976,78.741152,79.665635,80.749293,NaN,NaN,NaN
3,Arab World,ARB,"Access to electricity, urban (% of urban popul...",EG.ELC.ACCS.UR.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,95.939242,95.962166,96.352930,95.997833,96.649916,96.834184,97.003974,NaN,NaN,NaN
4,Arab World,ARB,Account ownership at a financial institution o...,FX.OWN.TOTL.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,22.260538,NaN,NaN,30.277130,NaN,NaN,37.165211,NaN,NaN,NaN


In [8]:
df_world.loc[
    (df_world["Country Name"] == "Spain")
    & (df_world["Indicator Name"] == "Population, male")
]["2018"].values + df_world.loc[
    (df_world["Country Name"] == "Spain")
    & (df_world["Indicator Name"] == "Population, female")
][
    "2018"
].values

array([46723749.])

Veo que el valor del dataframe original era correcto.

In [9]:
df_population["2019"].isna().sum()

2

In [10]:
df_population[df_population["2019"].isna()]

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,Unnamed: 64
67,Eritrea,ERI,"Population, total",SP.POP.TOTL,1007590.0,1033328.0,1060486.0,1088854.0,1118159.0,1148189.0,...,3213972.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
108,Not classified,INX,"Population, total",SP.POP.TOTL,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
df_population.loc[0, df_population.iloc[0].last_valid_index()]

106314.0

Veo que realmente tengo casi todos los datos de 2019 por lo que no hace falta que haga nada más...

In [12]:
# df["population"].fillna(value=df_population['2019'], inplace=True)
# df

Veo como ahora no tengo ningun nan en población.

# ML

## DEPARTURES

Completo los nan con un modelo de ML.

In [21]:
df_model = df[
    [
        "total_pop",
        "arrivals",
        "departures"
    ]
]

df_model

,total_pop,arrivals,departures
0,37172386.0,NaN,NaN
1,2866376.0,5340000.0,5415000.0
2,42228429.0,2657000.0,5610000.0
3,64948.0,20200.0,NaN
4,1198575.0,3042000.0,NaN
...,...,...,...
213,106977.0,381000.0,NaN
214,4569087.0,606000.0,NaN
215,28498687.0,366700.0,NaN
216,17351822.0,1072000.0,NaN


In [22]:
df_model_train_test = df_model.loc[~df_model["departures"].isna()]
df_model_predict = df_model.loc[df_model["departures"].isna()]

display(df_model_train_test)
display(df_model_predict)

,total_pop,arrivals,departures
1,2866376.0,5340000.0,5415000.0
2,42228429.0,2657000.0,5610000.0
5,30809762.0,218000.0,3000.0
7,44494502.0,6942000.0,11130000.0
8,2951776.0,1652000.0,1623000.0
...,...,...,...
207,327167434.0,79745920.0,92564000.0
208,3449299.0,3469000.0,1947000.0
210,292680.0,116000.0,29000.0
211,28870195.0,427000.0,1079000.0


,total_pop,arrivals,departures
0,37172386.0,NaN,NaN
3,64948.0,20200.0,NaN
4,1198575.0,3042000.0,NaN
6,96286.0,269000.0,NaN
10,105845.0,1082000.0,NaN
...,...,...,...
212,95540395.0,15498000.0,NaN
213,106977.0,381000.0,NaN
214,4569087.0,606000.0,NaN
215,28498687.0,366700.0,NaN


In [23]:
X_train_test = df_model_train_test.drop("departures", axis=1)
y_train_test = df_model_train_test["departures"]
display(X_train_test)
display(y_train_test)

,total_pop,arrivals
1,2866376.0,5340000.0
2,42228429.0,2657000.0
5,30809762.0,218000.0
7,44494502.0,6942000.0
8,2951776.0,1652000.0
...,...,...
207,327167434.0,79745920.0
208,3449299.0,3469000.0
210,292680.0,116000.0
211,28870195.0,427000.0


1       5415000.0
2       5610000.0
5          3000.0
7      11130000.0
8       1623000.0
          ...    
207    92564000.0
208     1947000.0
210       29000.0
211     1079000.0
217     2987000.0
Name: departures, Length: 129, dtype: float64

In [24]:
X_train, X_test, y_train, y_test = train_test_split(
    X_train_test, y_train_test, random_state=42
)

In [25]:
X_train_test.isna().sum()

total_pop    0
arrivals     0
dtype: int64

In [26]:
# categorical_pipe = Pipeline(
#     [
#         ("imputer", SimpleImputer()),
#     ]
# )

# preprocessing = ColumnTransformer([("cat", categorical_pipe, ["continent_code"])])

rnd_reg = Pipeline(
    [
        ("preprocess", SimpleImputer(strategy="median")),
        ("regressor", RandomForestRegressor(random_state=42)),
    ]
)

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start=200, stop=2000, num=10)]
# Number of features to consider at every split
max_features = ["auto", "sqrt"]
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num=11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {
    "regressor__n_estimators": n_estimators,
    "regressor__max_features": max_features,
    "regressor__max_depth": max_depth,
    "regressor__min_samples_split": min_samples_split,
    "regressor__min_samples_leaf": min_samples_leaf,
    "regressor__bootstrap": bootstrap,
}


gridsearch = RandomizedSearchCV(
    rnd_reg, random_grid, random_state=42, n_iter=500, n_jobs=-1, verbose=2)
gridsearch.fit(X_train_test, y_train_test)

Fitting 5 folds for each of 500 candidates, totalling 2500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   14.0s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:   32.1s
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:   57.6s
[Parallel(n_jobs=-1)]: Done 997 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 1442 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 1969 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done 2500 out of 2500 | elapsed:  4.1min finished


RandomizedSearchCV(estimator=Pipeline(steps=[('preprocess',
                                              SimpleImputer(strategy='median')),
                                             ('regressor',
                                              RandomForestRegressor(random_state=42))]),
                   n_iter=500, n_jobs=-1,
                   param_distributions={'regressor__bootstrap': [True, False],
                                        'regressor__max_depth': [10, 20, 30, 40,
                                                                 50, 60, 70, 80,
                                                                 90, 100, 110,
                                                                 None],
                                        'regressor__max_features': ['auto',
                                                                    'sqrt'],
                                        'regressor__min_samples_leaf': [1, 2,
                                              

In [27]:
gridsearch.best_params_

{'regressor__n_estimators': 200,
 'regressor__min_samples_split': 10,
 'regressor__min_samples_leaf': 4,
 'regressor__max_features': 'sqrt',
 'regressor__max_depth': 30,
 'regressor__bootstrap': True}

In [28]:
print(gridsearch.score(X_test, y_test))

y_predicted = gridsearch.predict(X_test)
print(mean_absolute_error(y_test, y_predicted))

0.6196480895766805
7226401.907441416


In [29]:
df_diff = pd.DataFrame({"predicted": y_predicted.astype(int), "real": y_test})
df_diff["diff"] = df_diff["real"] - df_diff["predicted"]
df_diff["percentatge_diff"] = (
    (df_diff["real"] - df_diff["predicted"]) / df_diff["real"]
) * 100
df_diff

,predicted,real,diff,percentatge_diff
92,27566059,9468000.0,-18098059.0,-191.149757
67,779699,174000.0,-605699.0,-348.102874
37,23542358,26033000.0,2490642.0,9.567249
53,9008822,7390000.0,-1618822.0,-21.905575
192,37327636,9966000.0,-27361636.0,-274.549829
93,11812437,7243000.0,-4569437.0,-63.087629
117,1916369,2244000.0,327631.0,14.600312
176,2939647,3188000.0,248353.0,7.790245
141,17487468,22115000.0,4627532.0,20.924856
46,1624822,180000.0,-1444822.0,-802.678889


In [30]:
gridsearch.best_score_

0.41850932240002414

In [31]:
pd.DataFrame(gridsearch.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_regressor__n_estimators,param_regressor__min_samples_split,param_regressor__min_samples_leaf,param_regressor__max_features,param_regressor__max_depth,param_regressor__bootstrap,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.296786,0.015317,0.022110,0.006321,400,5,1,sqrt,30,True,"{'regressor__n_estimators': 400, 'regressor__m...",0.529946,0.186593,-0.318922,0.684436,0.466777,0.309766,0.353250,158
1,1.332210,0.045106,0.077281,0.001990,2000,5,1,sqrt,10,True,"{'regressor__n_estimators': 2000, 'regressor__...",0.543699,0.183665,-0.344535,0.664556,0.466358,0.302748,0.360246,169
2,0.580752,0.017478,0.050809,0.004017,1200,5,2,sqrt,10,False,"{'regressor__n_estimators': 1200, 'regressor__...",0.594273,0.112425,-1.509589,0.769295,0.515454,0.096371,0.831352,356
3,0.969635,0.037739,0.078802,0.003495,2000,2,4,auto,30,False,"{'regressor__n_estimators': 2000, 'regressor__...",0.497400,0.044940,-3.980299,0.222806,0.420511,-0.558928,1.717945,477
4,1.058361,0.040226,0.064171,0.003462,1600,2,4,sqrt,10,True,"{'regressor__n_estimators': 1600, 'regressor__...",0.396779,0.337791,0.022962,0.704820,0.490718,0.390614,0.222145,38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,0.699524,0.012646,0.059307,0.001679,1400,10,2,sqrt,70,False,"{'regressor__n_estimators': 1400, 'regressor__...",0.556231,0.190208,-0.849861,0.777164,0.582716,0.251292,0.582347,204
496,1.182057,0.080786,0.076202,0.004354,1600,5,1,sqrt,20,True,"{'regressor__n_estimators': 1600, 'regressor__...",0.549469,0.186154,-0.369328,0.670968,0.463674,0.300187,0.370837,180
497,0.773292,0.131098,0.043919,0.002817,1000,5,4,sqrt,100,True,"{'regressor__n_estimators': 1000, 'regressor__...",0.398344,0.337812,0.000110,0.704981,0.491965,0.386643,0.229940,44
498,0.328094,0.038959,0.019339,0.001146,400,5,2,sqrt,80,True,"{'regressor__n_estimators': 400, 'regressor__m...",0.473626,0.221834,-0.169326,0.684548,0.493340,0.340804,0.294433,97


Realmente el gridsearch habia hecho un refit con todos los datos, por lo que los resultado anteriores son sobre datos que ya habia visto antes...

Pruebo como se comporta ante datos que no haya visto:

In [32]:
rnd_reg = Pipeline(
    [
        ("preprocess", SimpleImputer(strategy="median")),
        ("regressor", RandomForestRegressor(n_estimators=200, random_state=42)),
    ]
)

rnd_reg.fit(X_train, y_train)
print(rnd_reg.score(X_train, y_train))
print(rnd_reg.score(X_test, y_test))

y_predicted = rnd_reg.predict(X_train)
print(mean_absolute_error(y_train, y_predicted))
y_predicted = rnd_reg.predict(X_test)
print(mean_absolute_error(y_test, y_predicted))

0.8807692826990061
0.34301438309636756
2734205.1614583335
9532198.333333334


In [33]:
gridsearch.best_params_

{'regressor__n_estimators': 200,
 'regressor__min_samples_split': 10,
 'regressor__min_samples_leaf': 4,
 'regressor__max_features': 'sqrt',
 'regressor__max_depth': 30,
 'regressor__bootstrap': True}

In [34]:
rnd_reg = Pipeline(
    [
        ("preprocess", SimpleImputer(strategy="median")),
        (
            "regressor",
            RandomForestRegressor(
                n_estimators=200,
                min_samples_split=10,
                min_samples_leaf=1,
                max_features="sqrt",
                max_depth=20,
                bootstrap=True,
                random_state=42,
            ),
        ),
    ]
)


rnd_reg.fit(X_train, y_train)
print(rnd_reg.score(X_train, y_train))
print(rnd_reg.score(X_test, y_test))

y_predicted = rnd_reg.predict(X_train)
print(mean_absolute_error(y_train, y_predicted))
y_predicted = rnd_reg.predict(X_test)
print(mean_absolute_error(y_test, y_predicted))

0.7005276344116163
0.47777990221879785
4783497.894188573
8371516.59496566


Veo que he obtenido unos resultados bastantes buenos sin incluir la variable categorica del continente... me empeoraba mucho el resultado.

Hago la prediccion de los valores que no tengo:

## pruebo un treeregressor

In [43]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = Pipeline(
    [
        ("preprocess", SimpleImputer(strategy="median")),
        ("regressor", DecisionTreeRegressor(random_state=42))
    ]
)

tree_reg.fit(X_train_test, y_train_test)
print(tree_reg.score(X_train, y_train))
print(tree_reg.score(X_test, y_test))

y_predicted = tree_reg.predict(X_train)
print(mean_absolute_error(y_train, y_predicted))
y_predicted = tree_reg.predict(X_test)
print(mean_absolute_error(y_test, y_predicted))

1.0
1.0
0.0
0.0


In [44]:
tree_reg.fit(X_train, y_train)
print(tree_reg.score(X_train, y_train))
print(tree_reg.score(X_test, y_test))

y_predicted = tree_reg.predict(X_train)
print(mean_absolute_error(y_train, ytree_reg.fit(X_train_test, y_train_test)
print(tree_reg.score(X_train, y_train))
print(tree_reg.score(X_test, y_test))

y_predicted = tree_reg.predict(X_train)
print(mean_absolute_error(y_train, y_predicted))
y_predicted = tree_reg.predict(X_test)
print(mean_absolute_error(y_test, y_predicted))_predicted))
y_predicted = tree_reg.predict(X_test)
print(mean_absolute_error(y_test, y_predicted))

1.0
-0.06335849417858186
0.0
11197209.090909092


Vemos que el arbol generaliza muy mal.

In [221]:
df_model_predict

,total_pop,arrivals,departures,longitude,latitude,arrivals/total,arrivals/population


In [222]:
X_predict = df_model_predict.drop("departures", axis=1)

In [223]:
y_predicted = gridsearch.predict(X_predict)

ValueError: Found array with 0 sample(s) (shape=(0, 6)) while a minimum of 1 is required.

In [137]:
df_model_predict["predicted_departures"] = y_predicted
df_model_predict

<ipython-input-137-17947f4aa98d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_model_predict["predicted_departures"] = y_predicted


,total_pop,arrivals,departures,arrivals/total,arrivals/population,predicted_departures
0,37172386.0,NaN,NaN,NaN,NaN,3481835.0
3,64948.0,20200.0,NaN,0.000014,0.311018,33047.5
4,1198575.0,3042000.0,NaN,0.002141,2.538014,1831405.0
6,96286.0,269000.0,NaN,0.000189,2.793760,98665.0
10,105845.0,1082000.0,NaN,0.000762,10.222495,1287135.0
...,...,...,...,...,...,...
212,95540395.0,15498000.0,NaN,0.010909,0.162214,23303965.0
213,106977.0,381000.0,NaN,0.000268,3.561513,526725.0
214,4569087.0,606000.0,NaN,0.000427,0.132630,664490.0
215,28498687.0,366700.0,NaN,0.000258,0.012867,513035.0


In [138]:
df.head()

,country_name,country_code,total_pop,arrivals,departures,latitude,longitude,arrivals/total,departures/total,arrivals/population,departures/population,continent_name,continent_code,destinations
0,Afghanistan,AFG,37172386.0,NaN,NaN,33.0000,65.0,NaN,NaN,NaN,NaN,Asia,AS,"[IND, PAK, ARE, IRN, TUR, AZE, KWT, TJK]"
1,Albania,ALB,2866376.0,5340000.0,5415000.0,41.0000,20.0,0.003759,0.004280,1.862980,1.889145,Europe,EU,"[GRC, ITA, GBR, DEU, SVN, AUT, TUR]"
2,Algeria,DZA,42228429.0,2657000.0,5610000.0,28.0000,3.0,0.001870,0.004434,0.062920,0.132849,Africa,AF,"[DEU, FRA, TUR, CIV, ESP, JOR, LBN, MLI, BEL, ..."
3,American Samoa,ASM,64948.0,20200.0,NaN,-14.3333,-170.0,0.000014,NaN,0.311018,NaN,Oceania,OC,[USA]
4,Andorra,AND,1198575.0,3042000.0,NaN,42.5000,1.6,0.002141,NaN,2.538014,NaN,Europe,EU,NaN


In [139]:
df["departures"].fillna(
    value=df_model_predict["predicted_departures"], inplace=True)

In [141]:
df["departures/total"] = df["departures"] / df["departures"].sum()
df["departures/population"] = df["departures"] / df["total_pop"]

## Ahora completo los nan de arrivals

In [142]:
df.isna().sum()

country_name              0
country_code              0
total_pop                 0
arrivals                 10
departures                0
latitude                  0
longitude                 0
arrivals/total           10
departures/total          0
arrivals/population      10
departures/population     0
continent_name            0
continent_code           31
destinations              6
dtype: int64

In [146]:
df_model = df[
    [
        "total_pop",
        "arrivals",
        "latitude",
        "longitude",
        "departures",
        "departures/total",
        "departures/population",
    ]
]
df_model

,total_pop,arrivals,departures,departures/total,departures/population
0,37172386.0,NaN,3481835.0,0.002287,0.093667
1,2866376.0,5340000.0,5415000.0,0.003556,1.889145
2,42228429.0,2657000.0,5610000.0,0.003684,0.132849
3,64948.0,20200.0,33047.5,0.000022,0.508830
4,1198575.0,3042000.0,1831405.0,0.001203,1.527985
...,...,...,...,...,...
213,106977.0,381000.0,526725.0,0.000346,4.923722
214,4569087.0,606000.0,664490.0,0.000436,0.145432
215,28498687.0,366700.0,513035.0,0.000337,0.018002
216,17351822.0,1072000.0,713465.0,0.000469,0.041118


In [147]:
df_model.isna().sum()

total_pop                 0
arrivals                 10
departures                0
departures/total          0
departures/population     0
dtype: int64

In [148]:
df_model_train_test = df_model.loc[~df_model["arrivals"].isna()]
df_model_predict = df_model.loc[df_model["arrivals"].isna()]

display(df_model_train_test)
display(df_model_predict)

X_train_test = df_model_train_test.drop("arrivals", axis=1)
y_train_test = df_model_train_test["arrivals"]
display(X_train_test)
display(y_train_test)

,total_pop,arrivals,departures,departures/total,departures/population
1,2866376.0,5340000.0,5415000.0,0.003556,1.889145
2,42228429.0,2657000.0,5610000.0,0.003684,0.132849
3,64948.0,20200.0,33047.5,0.000022,0.508830
4,1198575.0,3042000.0,1831405.0,0.001203,1.527985
5,30809762.0,218000.0,3000.0,0.000002,0.000097
...,...,...,...,...,...
213,106977.0,381000.0,526725.0,0.000346,4.923722
214,4569087.0,606000.0,664490.0,0.000436,0.145432
215,28498687.0,366700.0,513035.0,0.000337,0.018002
216,17351822.0,1072000.0,713465.0,0.000469,0.041118


,total_pop,arrivals,departures,departures/total,departures/population
0,37172386.0,NaN,3481835.0,0.002287,0.093667
61,1308974.0,NaN,3004165.0,0.001973,2.295053
66,361313.0,NaN,2947335.0,0.001936,8.157290
77,117606.0,NaN,2947335.0,0.001936,25.061094
79,51709098.0,NaN,3793550.0,0.002491,0.073363
96,2786844.0,NaN,3106660.0,0.002040,1.114759
106,25549819.0,NaN,3052220.0,0.002005,0.119462
114,4818977.0,NaN,3712485.0,0.002438,0.770389
139,32510453.0,NaN,3223555.0,0.002117,0.099154
178,15008154.0,NaN,2720290.0,0.001787,0.181254


,total_pop,departures,departures/total,departures/population
1,2866376.0,5415000.0,0.003556,1.889145
2,42228429.0,5610000.0,0.003684,0.132849
3,64948.0,33047.5,0.000022,0.508830
4,1198575.0,1831405.0,0.001203,1.527985
5,30809762.0,3000.0,0.000002,0.000097
...,...,...,...,...
213,106977.0,526725.0,0.000346,4.923722
214,4569087.0,664490.0,0.000436,0.145432
215,28498687.0,513035.0,0.000337,0.018002
216,17351822.0,713465.0,0.000469,0.041118


1      5340000.0
2      2657000.0
3        20200.0
4      3042000.0
5       218000.0
         ...    
213     381000.0
214     606000.0
215     366700.0
216    1072000.0
217    2580000.0
Name: arrivals, Length: 201, dtype: float64

In [152]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start=200, stop=2000, num=10)]
# Number of features to consider at every split
max_features = ["auto", "sqrt"]
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num=11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {
    "regressor__n_estimators": n_estimators,
    "regressor__max_features": max_features,
    "regressor__max_depth": max_depth,
    "regressor__min_samples_split": min_samples_split,
    "regressor__min_samples_leaf": min_samples_leaf,
    "regressor__bootstrap": bootstrap,
}

In [153]:
rnd_reg = Pipeline(
    [
        ("preprocess", SimpleImputer(strategy="median")),
        ("regressor", RandomForestRegressor(random_state=42)),
    ]
)

gridsearch = RandomizedSearchCV(
    rnd_reg, random_grid, n_iter=100, n_jobs=-1, verbose=2)
gridsearch.fit(X_train_test, y_train_test)
gridsearch.best_params_

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    3.6s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   15.3s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:   36.1s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:   51.5s finished


{'regressor__n_estimators': 600,
 'regressor__min_samples_split': 2,
 'regressor__min_samples_leaf': 2,
 'regressor__max_features': 'sqrt',
 'regressor__max_depth': 70,
 'regressor__bootstrap': True}

In [154]:
print(gridsearch.score(X_test, y_test))

y_predicted = gridsearch.predict(X_test)
print(mean_absolute_error(y_test, y_predicted))

0.5125973955375416
8568942.670446169


In [155]:
rnd_reg = Pipeline(
    [
        ("preprocess", SimpleImputer(strategy="median")),
        (
            "regressor",
            RandomForestRegressor(
                n_estimators=400,
                min_samples_split=10,
                min_samples_leaf=4,
                max_features="sqrt",
                max_depth=100,
                bootstrap=True,
                random_state=42,
            ),
        ),
    ]
)

# rnd_reg = Pipeline(
#     [
#         ("preprocess", SimpleImputer(strategy="median")),
#         (
#             "regressor",
#             RandomForestRegressor(
#                 n_estimators=800,
#                 min_samples_split=5,
#                 min_samples_leaf=4,
#                 max_features="sqrt",
#                 max_depth=40,
#                 bootstrap=True,
#                 random_state=42,
#             ),
#         ),
#     ]
# )

rnd_reg.fit(X_train, y_train)
print(rnd_reg.score(X_train, y_train))
print(rnd_reg.score(X_test, y_test))

y_predicted = rnd_reg.predict(X_train)
print(mean_absolute_error(y_train, y_predicted))
y_predicted = rnd_reg.predict(X_test)
print(mean_absolute_error(y_test, y_predicted))

0.4803971046517477
0.5489940870596127
5534681.976072473
7912602.057046815


In [156]:
df_model_predict

,total_pop,arrivals,departures,departures/total,departures/population
0,37172386.0,NaN,3481835.0,0.002287,0.093667
61,1308974.0,NaN,3004165.0,0.001973,2.295053
66,361313.0,NaN,2947335.0,0.001936,8.157290
77,117606.0,NaN,2947335.0,0.001936,25.061094
79,51709098.0,NaN,3793550.0,0.002491,0.073363
96,2786844.0,NaN,3106660.0,0.002040,1.114759
106,25549819.0,NaN,3052220.0,0.002005,0.119462
114,4818977.0,NaN,3712485.0,0.002438,0.770389
139,32510453.0,NaN,3223555.0,0.002117,0.099154
178,15008154.0,NaN,2720290.0,0.001787,0.181254


In [157]:
X_predict = df_model_predict.drop("arrivals", axis=1)

In [158]:
y_predicted = gridsearch.predict(X_predict)
df_model_predict["predicted_arrivals"] = y_predicted
df_model_predict

<ipython-input-158-bac6fb75c463>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_model_predict["predicted_arrivals"] = y_predicted


,total_pop,arrivals,departures,departures/total,departures/population,predicted_arrivals
0,37172386.0,NaN,3481835.0,0.002287,0.093667,4.204120e+06
61,1308974.0,NaN,3004165.0,0.001973,2.295053,6.306666e+06
66,361313.0,NaN,2947335.0,0.001936,8.157290,5.737506e+06
77,117606.0,NaN,2947335.0,0.001936,25.061094,5.727868e+06
79,51709098.0,NaN,3793550.0,0.002491,0.073363,4.982554e+06
96,2786844.0,NaN,3106660.0,0.002040,1.114759,4.342403e+06
106,25549819.0,NaN,3052220.0,0.002005,0.119462,4.077287e+06
114,4818977.0,NaN,3712485.0,0.002438,0.770389,3.261130e+06
139,32510453.0,NaN,3223555.0,0.002117,0.099154,4.224430e+06
178,15008154.0,NaN,2720290.0,0.001787,0.181254,7.455549e+06


In [161]:
df["arrivals"].fillna(
    value=df_model_predict["predicted_arrivals"], inplace=True)
df["arrivals/total"] = df["arrivals"] / df["arrivals"].sum()
df["arrivals/population"] = df["arrivals"] / df["total_pop"]
df

,country_name,country_code,total_pop,arrivals,departures,latitude,longitude,arrivals/total,departures/total,arrivals/population,departures/population,continent_name,continent_code,destinations
0,Afghanistan,AFG,37172386.0,4.204120e+06,3481835.0,33.0000,65.0000,0.002858,0.002287,0.113098,0.093667,Asia,AS,"[IND, PAK, ARE, IRN, TUR, AZE, KWT, TJK]"
1,Albania,ALB,2866376.0,5.340000e+06,5415000.0,41.0000,20.0000,0.003630,0.003556,1.862980,1.889145,Europe,EU,"[GRC, ITA, GBR, DEU, SVN, AUT, TUR]"
2,Algeria,DZA,42228429.0,2.657000e+06,5610000.0,28.0000,3.0000,0.001806,0.003684,0.062920,0.132849,Africa,AF,"[DEU, FRA, TUR, CIV, ESP, JOR, LBN, MLI, BEL, ..."
3,American Samoa,ASM,64948.0,2.020000e+04,33047.5,-14.3333,-170.0000,0.000014,0.000022,0.311018,0.508830,Oceania,OC,[USA]
4,Andorra,AND,1198575.0,3.042000e+06,1831405.0,42.5000,1.6000,0.002068,0.001203,2.538014,1.527985,Europe,EU,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213,Virgin Islands (U.S.),VIR,106977.0,3.810000e+05,526725.0,18.3333,-64.8333,0.000259,0.000346,3.561513,4.923722,North America,NaN,"[VGB, PRI, USA, AIA, DMA, KNA]"
214,West Bank and Gaza,PSE,4569087.0,6.060000e+05,664490.0,32.0000,35.2500,0.000412,0.000436,0.132630,0.145432,Asia,AS,NaN
215,"Yemen, Rep.",YEM,28498687.0,3.667000e+05,513035.0,15.0000,48.0000,0.000249,0.000337,0.012867,0.018002,Asia,AS,"[ARE, SAU, SOM, DJI, BHR, JOR, IND, EGY, QAT, ..."
216,Zambia,ZMB,17351822.0,1.072000e+06,713465.0,-15.0000,30.0000,0.000729,0.000469,0.061780,0.041118,Africa,AF,"[NLD, ZWE, MWI, KEN, ZAF, BWA, AGO, ARE, ETH, ..."


In [162]:
df.isna().sum()

country_name              0
country_code              0
total_pop                 0
arrivals                  0
departures                0
latitude                  0
longitude                 0
arrivals/total            0
departures/total          0
arrivals/population       0
departures/population     0
continent_name            0
continent_code           31
destinations              6
dtype: int64

De que paises no tengo la poblacion??

In [164]:
df.loc[df["total_pop"].isna()]

,country_name,country_code,total_pop,arrivals,departures,latitude,longitude,arrivals/total,departures/total,arrivals/population,departures/population,continent_name,continent_code,destinations
